<a href="https://colab.research.google.com/github/JayChaoNTPUACC/Early-Warning-For-Stocks-With-Abnormal-Trading-Conditions-Based-On-Generative-Adversarial-Networks/blob/main/GetData/getdata.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
#安裝套件
!pip install ffn
!pip install FinMind
url = 'https://launchpad.net/~mario-mariomedina/+archive/ubuntu/talib/+files'
ext = '0.4.0-oneiric1_amd64.deb -qO'
!wget $url/libta-lib0_$ext libta.deb
!wget $url/ta-lib0-dev_$ext ta.deb
!dpkg -i libta.deb ta.deb
!pip install ta-lib

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.1/49.1 kB 928.2 kB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 147.7/147.7 kB 5.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.5/62.5 kB 6.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 27.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 137.9/137.9 kB 13.8 MB/s eta 0:00:00
  Created wheel for ta: filename=ta-0.5.25-py3-none-any.whl size=24862 sha256=da0301c82cb79b85225c372f6c20cd3bc0991ff6c508d2c0ba5fa72d0c2dd5ce
  Stored in directory: /root/.cache/pip/wheels/d4/bd/c4/0dda911b5461ee856352cd8c9e16472229bb51c95e74b74c08
Successfully built ta
Selecting previously unselected package libta-lib0.
(Reading database ... 120880 files and directories currently installed.)
Preparing to unpack libta.deb ...
Unpacking libta-lib0 (0.4.0-oneiric1) ...
Selecting previously unselected package ta-lib0-dev.
Preparing to unpack ta.deb 

In [2]:
#import套件
import os
import pandas as pd
import ffn
import FinMind
import talib
from talib import abstract
import requests
from datetime import datetime,timedelta

In [3]:
#讀取資料
df_punishdata = pd.read_excel("punish.xlsx")
#將證券代號型別轉為字串
df_punishdata["證券代號"] = df_punishdata["證券代號"].astype(str)

In [4]:
df_punishdata

,公布日期,證券代號,證券名稱,處置條件,代碼,處置開始時間,處置結束時間
0,112/03/24,1435,中福,連續三次,1,112/03/27,112/04/12
1,112/06/28,1471,首利,連續三次,1,112/06/29,112/07/12
2,112/06/29,1519,華城,最近三十個營業日已有十二次,6,112/06/30,112/07/13
3,112/04/19,1529,樂事綠能,連續三次,1,112/04/20,112/05/04
4,112/04/28,1727,中華化,連續三次,1,112/05/02,112/05/15
5,112/04/20,1810,和成,連續三次,1,112/04/21,112/05/05
6,112/06/28,2059,川湖,連續三次,1,112/06/29,112/07/12
7,112/07/21,2316,楠梓電,連續三次,1,112/07/24,112/08/04
8,112/07/31,2329,華泰,連續三次,1,112/08/01,112/08/14
9,112/07/19,2356,英業達,連續三次,1,112/07/20,112/08/02


In [5]:
# 以下幾個functions到時候再封裝成物件
# talib MACD is the moving average covergence divergence trend-following momentum indicator.
def ReturnMACD(close,fastperiod=12, slowperiod=26,signalperiod=9):
    return talib.MACD(close,fastperiod=fastperiod,slowperiod=slowperiod,signalperiod=signalperiod)

In [6]:
# talib RSI is the relative strength index momentum oscillator that measures the speed and change of price movements.
def ReturnRSI(close,timeperiod=10):
    return talib.RSI(close, timeperiod=timeperiod)

In [7]:
# talib BOLL are Billinger bands given by adding and subtrating the standard deviation on a simple moving average.
def ReturnBBANDS(close, timeperiod=5, matype=1):
    return abstract.BBANDS(close, timeperiod, matype)

In [8]:
# talib CCI is the comodity channel index momentum oscillator used to determine overbought and oversold levels.
def ReturnCCI(high, low, close, timeperiod=10):
    return talib.CCI(high, low, close, timeperiod)

In [9]:
# talib DX is the directional movement index, a momentum indicator that calculates the strength of the upward or downward trend over a given period.
def ReturnDX(high, low, close, timeperiod=10):
    return talib.DX(high, low, close, timeperiod=10)

In [10]:
# talib sma is a simple moving average.
def ReturnSMA(close,timeperiod=5):
    return talib.SMA(close, timeperiod)

In [11]:
i = 1
for index, row in df_punishdata.iterrows():
  stock_num = row["證券代號"]
  announced_day = str(int(row["公布日期"].split('/')[0])+1911) + "-" + str(row["公布日期"].split('/')[1]) + "-" + str(row["公布日期"].split('/')[2])
  # print(announced_day)
  stock_name = row["證券名稱"]
  stock_punishreason_kind = row["代碼"]
  punish_startdate = str(int(row["處置開始時間"].split('/')[0])+1911) + "-" + str(row["處置開始時間"].split('/')[1]) + "-" + str(row["處置開始時間"].split('/')[2])
  punish_enddate = str(int(row["處置結束時間"].split('/')[0])+1911) + "-" + str(row["處置結束時間"].split('/')[1]) + "-" + str(row["處置結束時間"].split('/')[2])
  url = "https://api.finmindtrade.com/api/v4/data"
  token = ""
  data_startdate = datetime.strftime(datetime.strptime(announced_day, '%Y-%m-%d') - timedelta(days = 182), '%Y-%m-%d')
  data_enddate = announced_day
  # print(data_startdate)
  # print(data_enddate)
  parameter = {
      "dataset": "TaiwanStockPrice",
      "data_id": stock_num,
      "start_date": data_startdate,
      "end_date": data_enddate,
      "token": token
  }
  get_stock_deal_info = pd.DataFrame(requests.get(url, params=parameter).json()["data"])
  print(get_stock_deal_info)
  print(get_stock_deal_info.close)
  print(ReturnMACD(get_stock_deal_info.close,12,26,9))
  #為何會排除1435是因為抓到的筆數都只有5筆無法產生技術指標
  if stock_num != "1435":
      #MACD部分 以下各部分到時候都可以封裝成物件的方式處理，精簡程式碼
      MACD_macd,MACD_macdsignal,MACD_macdhist = ReturnMACD(get_stock_deal_info.close,12,26,9)
      MACD_macd.name = "MACD_macd"
      MACD_macdsignal.name = "MACD_macdsignal"
      MACD_macdhist.name = "MACD_macdhist"
      # print(MACD_macd)
      # print(type(MACD_macd))
      get_stock_deal_info = pd.concat([get_stock_deal_info, MACD_macd], axis=1)
      get_stock_deal_info = pd.concat([get_stock_deal_info, MACD_macdsignal], axis=1)
      get_stock_deal_info = pd.concat([get_stock_deal_info, MACD_macdhist], axis=1)
      # print(get_stock_deal_info)

      # RSI
      RSI10 = ReturnRSI(get_stock_deal_info.close,10)
      RSI10.name = "RSI10"
      get_stock_deal_info = pd.concat([get_stock_deal_info,RSI10], axis = 1)

      # BOLL
      BOLL_Upper, BOLL_Middle, BOLL_Lower = talib.BBANDS(get_stock_deal_info.close,5,1)
      BOLL_Upper.name = "BOLL_Upper"
      BOLL_Middle.name = "BOLL_Middle"
      BOLL_Lower.name = "BOLL_Lower"
      get_stock_deal_info = pd.concat([get_stock_deal_info, BOLL_Upper], axis=1)
      get_stock_deal_info = pd.concat([get_stock_deal_info, BOLL_Middle], axis=1)
      get_stock_deal_info = pd.concat([get_stock_deal_info, BOLL_Lower], axis=1)

      # CCI
      CCI10 = ReturnCCI(get_stock_deal_info["max"].values, get_stock_deal_info["min"].values, get_stock_deal_info["close"].values,10)
      get_stock_deal_info = pd.concat([get_stock_deal_info, pd.DataFrame(CCI10,columns=["CCI10"])], axis = 1)

      # DX
      DX10 = ReturnDX(get_stock_deal_info["max"].values, get_stock_deal_info["min"].values, get_stock_deal_info["close"].values,10)
      get_stock_deal_info = pd.concat([get_stock_deal_info, pd.DataFrame(DX10,columns=["DX10"])], axis = 1)

      # SMA
      SMA5 = ReturnSMA(get_stock_deal_info.close,5)
      SMA5.name = "SMA5"
      get_stock_deal_info = pd.concat([get_stock_deal_info,SMA5], axis = 1)
      SMA10 = ReturnSMA(get_stock_deal_info.close,10)
      SMA10.name = "SMA10"
      get_stock_deal_info = pd.concat([get_stock_deal_info,SMA10], axis = 1)
      SMA20 = ReturnSMA(get_stock_deal_info.close,20)
      SMA20.name = "SMA20"
      get_stock_deal_info = pd.concat([get_stock_deal_info,SMA20], axis = 1)

      # 另外一種比較簡便的作法，不過結果怪怪的，因此手動設上面的function並呼叫計算結果
      # # ta_list = talib.get_functions()
      # # ta_list = ['HT_DCPERIOD','HT_DCPHASE','HT_PHASOR','HT_SINE','HT_TRENDMODE','MINMAXINDEX','MACD','RSI','BBANDS','CCI','DX','ADX','STOCH']
      # ta_list = ['MACD',"RSI","BBANDS"]
      # # ta_list = ['HT_DCPERIOD', 'HT_DCPHASE', 'HT_PHASOR', 'HT_SINE', 'HT_TRENDMODE', 'MAX', 'MAXINDEX', 'MIN', 'MININDEX', 'MINMAX', 'MINMAXINDEX', 'SUM', 'ACOS', 'ASIN', 'ATAN', 'CEIL', 'COS', 'COSH', 'EXP', 'FLOOR', 'LN', 'LOG10', 'SIN', 'SINH', 'SQRT', 'TAN', 'TANH', 'APO', 'CMO', 'MACD', 'MACDEXT', 'MACDFIX', 'MOM', 'PPO', 'ROC', 'ROCP', 'ROCR', 'ROCR100', 'RSI', 'STOCHRSI', 'TRIX', 'BBANDS', 'DEMA', 'EMA', 'HT_TRENDLINE', 'KAMA', 'MA', 'MAMA', 'MIDPOINT', 'SMA', 'T3', 'TEMA', 'TRIMA', 'WMA', 'LINEARREG', 'LINEARREG_ANGLE', 'LINEARREG_INTERCEPT', 'LINEARREG_SLOPE', 'STDDEV', 'TSF', 'VAR']
      # for x in ta_list:
      #     try:
      #         output = eval('abstract.'+x+'(get_stock_deal_info)')
      #         print(output)
      #         #print(type(output))
      #         output.name = x.lower() if type(output) == pd.core.series.Series else None
      #         print(get_stock_deal_info.tail(20))
      #         print(pd.DataFrame(output).tail(20))
      #         get_stock_deal_info = pd.concat([get_stock_deal_info, pd.DataFrame(output)], axis=1)
      #         #print(get_stock_deal_info)
      #     except:
      #         print(x)
      #         # ta_list.remove(x)
      # print(ta_list)


      if i == 1:
        df_alldatas = get_stock_deal_info.tail(20)
      else:
        df_alldatas = pd.concat([df_alldatas, get_stock_deal_info.tail(20)], ignore_index=True)
        # # 測試抓取兩個股票資料用
        # if i == 2:
        #   break
      i += 1
df_alldatas


         date stock_id  Trading_Volume  Trading_money   open    max    min  \
0  2023-03-20     1435           67302        2700989  40.10  44.20  39.80   
1  2023-03-21     1435           55650        2105853  35.90  40.10  35.85   
2  2023-03-22     1435          161692        5674857  37.25  37.25  34.20   
3  2023-03-23     1435           42130        1427894  35.00  35.00  33.20   
4  2023-03-24     1435           44945        1473605  32.60  33.35  32.25   

   close  spread  Trading_turnover  
0  39.80    0.00               169  
1  38.00   -1.80               139  
2  35.35   -2.65               222  
3  34.15   -1.20               105  
4  32.25   -1.90               117  
0    39.80
1    38.00
2    35.35
3    34.15
4    32.25
Name: close, dtype: float64
(0   NaN
1   NaN
2   NaN
3   NaN
4   NaN
dtype: float64, 0   NaN
1   NaN
2   NaN
3   NaN
4   NaN
dtype: float64, 0   NaN
1   NaN
2   NaN
3   NaN
4   NaN
dtype: float64)
           date stock_id  Trading_Volume  Trading_money  

,date,stock_id,Trading_Volume,Trading_money,open,max,min,close,spread,Trading_turnover,...,MACD_macdhist,RSI10,BOLL_Upper,BOLL_Middle,BOLL_Lower,CCI10,DX10,SMA5,SMA10,SMA20
0,2023-05-30,1471,542514,4900735,9.17,9.17,9.00,9.05,-0.07,321,...,-0.046650,45.587036,9.226061,9.128,8.931878,-56.737589,13.650578,9.128,9.117,9.2285
1,2023-05-31,1471,431485,3927339,9.12,9.15,9.06,9.12,0.07,251,...,-0.039771,48.596308,9.150780,9.094,8.980439,-56.161616,13.650578,9.094,9.129,9.1880
2,2023-06-01,1471,461854,4218377,9.12,9.17,9.10,9.14,0.02,260,...,-0.032590,49.483227,9.137558,9.088,8.988884,-19.023987,11.265397,9.088,9.135,9.1565
3,2023-06-02,1471,2006001,18726302,9.27,9.80,9.19,9.58,0.44,805,...,0.001281,64.468957,9.393458,9.202,8.819085,241.506647,35.853231,9.202,9.189,9.1640
4,2023-06-05,1471,1881206,18051341,9.66,9.67,9.50,9.58,0.00,801,...,0.022023,64.468957,9.529423,9.294,8.823154,174.683544,35.853231,9.294,9.232,9.1710
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
975,2023-02-22,8996,15275215,3666810981,238.00,246.50,227.50,242.00,3.00,10027,...,0.225042,73.163264,241.222650,237.900,231.254701,100.214362,30.625927,237.900,235.350,222.5500
976,2023-02-23,8996,13890045,3293631596,242.00,245.00,233.00,234.00,-8.00,9645,...,-0.483204,60.817638,240.129286,236.700,229.841429,44.673540,30.625927,236.700,235.750,224.9000
977,2023-02-24,8996,10135285,2415411520,238.50,240.50,234.50,239.00,5.00,6194,...,-0.709288,64.927475,240.756810,238.000,232.486380,56.540084,30.625927,238.000,236.750,226.5750
978,2023-03-01,8996,12219884,2994677556,239.50,250.50,238.50,244.50,5.50,9066,...,-0.587785,68.912816,243.215679,239.700,232.668642,242.042042,50.041759,239.700,237.500,228.8000


In [12]:
df_alldatas.count()

date                980
stock_id            980
Trading_Volume      980
Trading_money       980
open                980
max                 980
min                 980
close               980
spread              980
Trading_turnover    980
MACD_macd           980
MACD_macdsignal     980
MACD_macdhist       980
RSI10               980
BOLL_Upper          980
BOLL_Middle         980
BOLL_Lower          980
CCI10               980
DX10                980
SMA5                980
SMA10               980
SMA20               980
dtype: int64

In [13]:
df_alldatas = pd.merge(df_alldatas,df_punishdata, left_on='stock_id', right_on='證券代號')

,date,stock_id,Trading_Volume,Trading_money,open,max,min,close,spread,Trading_turnover,...,RSI10,BOLL_Upper,BOLL_Middle,BOLL_Lower,CCI10,DX10,SMA5,SMA10,SMA20,代碼
0,2023-05-30,1471,542514,4900735,9.17,9.17,9.00,9.05,-0.07,321,...,45.587036,9.226061,9.128,8.931878,-56.737589,13.650578,9.128,9.117,9.2285,1
1,2023-05-31,1471,431485,3927339,9.12,9.15,9.06,9.12,0.07,251,...,48.596308,9.150780,9.094,8.980439,-56.161616,13.650578,9.094,9.129,9.1880,1
2,2023-06-01,1471,461854,4218377,9.12,9.17,9.10,9.14,0.02,260,...,49.483227,9.137558,9.088,8.988884,-19.023987,11.265397,9.088,9.135,9.1565,1
3,2023-06-02,1471,2006001,18726302,9.27,9.80,9.19,9.58,0.44,805,...,64.468957,9.393458,9.202,8.819085,241.506647,35.853231,9.202,9.189,9.1640,1
4,2023-06-05,1471,1881206,18051341,9.66,9.67,9.50,9.58,0.00,801,...,64.468957,9.529423,9.294,8.823154,174.683544,35.853231,9.294,9.232,9.1710,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1175,2023-02-24,8996,10135285,2415411520,238.50,240.50,234.50,239.00,5.00,6194,...,64.927475,240.756810,238.000,232.486380,56.540084,30.625927,238.000,236.750,226.5750,4
1176,2023-03-01,8996,12219884,2994677556,239.50,250.50,238.50,244.50,5.50,9066,...,68.912816,243.215679,239.700,232.668642,242.042042,50.041759,239.700,237.500,228.8000,3
1177,2023-03-01,8996,12219884,2994677556,239.50,250.50,238.50,244.50,5.50,9066,...,68.912816,243.215679,239.700,232.668642,242.042042,50.041759,239.700,237.500,228.8000,4
1178,2023-03-02,8996,5355469,1308148387,245.50,248.00,241.00,241.00,-3.50,4091,...,63.787759,243.627038,240.100,233.045923,142.857143,50.041759,240.100,238.250,230.2250,3


In [16]:
df_alldatas.drop(columns=["公布日期", "證券代號",	"證券名稱",	"處置條件",	"處置開始時間",	"處置結束時間"],inplace=True)

In [17]:
df_alldatas

,date,stock_id,Trading_Volume,Trading_money,open,max,min,close,spread,Trading_turnover,...,RSI10,BOLL_Upper,BOLL_Middle,BOLL_Lower,CCI10,DX10,SMA5,SMA10,SMA20,代碼
0,2023-05-30,1471,542514,4900735,9.17,9.17,9.00,9.05,-0.07,321,...,45.587036,9.226061,9.128,8.931878,-56.737589,13.650578,9.128,9.117,9.2285,1
1,2023-05-31,1471,431485,3927339,9.12,9.15,9.06,9.12,0.07,251,...,48.596308,9.150780,9.094,8.980439,-56.161616,13.650578,9.094,9.129,9.1880,1
2,2023-06-01,1471,461854,4218377,9.12,9.17,9.10,9.14,0.02,260,...,49.483227,9.137558,9.088,8.988884,-19.023987,11.265397,9.088,9.135,9.1565,1
3,2023-06-02,1471,2006001,18726302,9.27,9.80,9.19,9.58,0.44,805,...,64.468957,9.393458,9.202,8.819085,241.506647,35.853231,9.202,9.189,9.1640,1
4,2023-06-05,1471,1881206,18051341,9.66,9.67,9.50,9.58,0.00,801,...,64.468957,9.529423,9.294,8.823154,174.683544,35.853231,9.294,9.232,9.1710,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1175,2023-02-24,8996,10135285,2415411520,238.50,240.50,234.50,239.00,5.00,6194,...,64.927475,240.756810,238.000,232.486380,56.540084,30.625927,238.000,236.750,226.5750,4
1176,2023-03-01,8996,12219884,2994677556,239.50,250.50,238.50,244.50,5.50,9066,...,68.912816,243.215679,239.700,232.668642,242.042042,50.041759,239.700,237.500,228.8000,3
1177,2023-03-01,8996,12219884,2994677556,239.50,250.50,238.50,244.50,5.50,9066,...,68.912816,243.215679,239.700,232.668642,242.042042,50.041759,239.700,237.500,228.8000,4
1178,2023-03-02,8996,5355469,1308148387,245.50,248.00,241.00,241.00,-3.50,4091,...,63.787759,243.627038,240.100,233.045923,142.857143,50.041759,240.100,238.250,230.2250,3


In [18]:
output_file_name = 'get_stocks_result.csv'
df_alldatas.to_csv(output_file_name,sep=',',index=False,header=True)